In [ ]:
from IPython.display import Audio
import librosa as lr
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [ ]:
x, sr = lr.load('BT_Pipes_2.wav', sr=None, mono=False)
print x.shape

In [ ]:
Audio(x, rate=sr)

In [ ]:
X_l = lr.stft(x[0, :], n_fft=2048, hop_length=512)
X_r = lr.stft(x[1, :], n_fft=2048, hop_length=512)

# Add symmetry to vector (non vectorized implementation)

Needed for later median_filtering

In [ ]:
def add_symmetry(a, wsize=13):
    n = len(a)
    out = []
    for i in range(wsize + 1, 0, -1):
        out.append(a[i - 1])
    for i in range(n):
        out.append(a[i])
    for i in range(n, n - wsize + 1, -1):
        out.append(a[i - 1])
    return np.array(out)

In [ ]:
a = np.array(range(40))
wsize = 13
b = np.concatenate([a[wsize::-1], a, a[:-wsize:-1]])
print b
assert all(add_symmetry(a, wsize) == b)

# My implementation of HPSS using external median-filter module

Because `scipy.ndimage.median_filter` is used in librosa HPSS functions.

I've installed this one in the virtualenv:
https://github.com/craffel/median-filter

In [ ]:
import median_filter

In [ ]:
X_l_m, X_l_p = lr.core.magphase(X_l)

In [ ]:
X_l_m = X_l_m.astype(np.double)

In [ ]:
def horizontal_filtering(S, win_harm=321):
    harm = []
    for i in range(S.shape[0]):
        row = np.array(add_symmetry(S[i, :], win_harm))
        median_filter.filter(row, win_harm)
        harm.append(row[win_harm+1:-win_harm+1])
    return np.array(harm)

In [ ]:
my_harm = horizontal_filtering(X_l_m, 321)
harm = scipy.ndimage.median_filter(X_l_m, size=(1, 321), mode='reflect')

In [ ]:
assert np.sum(my_harm - harm) == 0.0
# Great, they are equal.

In [ ]:
def vertical_filtering(S, win_perc=321):
    S = S.T
    perc = []
    for i in range(S.shape[0]):
        col = np.array(add_symmetry(S[i, :], win_perc))
        median_filter.filter(col, win_perc)
        perc.append(col[win_perc+1:-win_perc+1])
    return np.array(perc).T

In [ ]:
my_perc = vertical_filtering(X_l_m, 321)
perc = scipy.ndimage.median_filter(X_l_m, size=(321, 1), mode='reflect')

In [ ]:
assert np.sum(my_perc - perc) == 0.0
# Great, they are equal.

In [ ]:
def my_soft_mask(X, X_ref, power=1, split_zeros=False):
    mask = np.empty_like(X)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            x = X[i, j]
            x_ref = X_ref[i, j]
            z = max(x, x_ref)
            if z < 1e-8:
                if split_zeros:
                    m = 0.5
                else:
                    m = 0.0
            else:
                m = (x / z) ** power
                rm = (x_ref / z) ** power
                m = m / (m + rm)
            mask[i, j] = m
    return mask


In [ ]:
def my_hpss(S, win_harm=31, win_perc=31, power=2.0, margin_harm=1.0, margin_perc=1.0):
    my_harm = horizontal_filtering(S, win_harm)
    my_perc = vertical_filtering(S, win_perc)
    split_zeros = (margin_harm == 1 and margin_perc == 1)
    mask_harm = my_soft_mask(my_harm, my_perc * margin_harm,
                             power=power, split_zeros=split_zeros)
    mask_perc = my_soft_mask(my_perc, my_harm * margin_perc,
                             power=power, split_zeros=split_zeros)
    return ((S * mask_harm), (S * mask_perc))

In [ ]:
H, P = lr.decompose.hpss(X_l_m, (131, 51), power=2.5, mask=False, margin=(1.0, 1.0))

In [ ]:
myH, myP = my_hpss(X_l_m, 131, 51, 2.5, 1.0, 1.0)

In [ ]:
assert np.sum(H - myH) == 0.0
assert np.sum(P - myP) == 0.0
# Great they are equal

## ISTFT

In [ ]:
y_h = lr.istft(myH * np.exp(1j*X_l_p))
lr.output.write_wav('harmonic.wav', y_h, 44100)

In [ ]:
y_p = lr.istft(myP * np.exp(1j*X_l_p))
lr.output.write_wav('percussive.wav', y_p, 44100)

In [ ]:
x_istft = lr.istft(X_l_m * np.exp(1j*X_l_p))
lr.output.write_wav('original_istft.wav', x_istft, 44100)

# Diff with C++ version

In [ ]:
import numpy as np
def load_file(path):
    x = np.fromfile(path, dtype=np.int32)
    size_x = (x[0], x[1])
    x = np.fromfile(path, dtype=np.float32)
    x = x[2:]
    x = x.reshape(size_x)
    return x

In [ ]:
x = load_file('./X.dat')
x_harm = load_file('./Xharm.dat')
x_perc = load_file('./Xperc.dat')

In [ ]:
plt.figure()
plt.imshow(np.log10(x[:100].T), origin='lower', aspect='auto')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.log10(x_harm[:100].T), origin='lower', aspect='auto')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.log10(x_perc[:100].T), origin='lower', aspect='auto')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.abs(X_r[:, :100]), origin='lower', aspect='auto')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.log10(X_l_m[:, :100]), origin='lower', aspect='auto')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.log10(my_harm[:, :100]), origin='lower', aspect='auto')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.log10(my_perc[:, :100]), origin='lower', aspect='auto')
plt.tight_layout()
plt.show()